# Make tables: calculate p-values and differential expressions for each cancer 

Create a dataframe with p-value results from t-tests for all proteins (trans proteomics when PTEN has cnv deletions compared to PTEN wildtype). The dataframe also includes the change in medians between deletions and wildtype. Prepare these tables for further analysis by creating csv files.

In [1]:
import pandas as pd
import numpy as np
import re

import cptac
import cptac.utils as u
import plot_utils as p

In [2]:
print('cptac version:', cptac.version())

cptac version: 0.8.5


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
'''
Params
cancer_object: Object. The loaded cancer data set.
all_prot: List. trans proteins used to get proteomics data. 
gene_in: String. Gene used to get mutation status for the Mutation col.
utils: utils package from cptac.  

Returns a dataframe with trans proteomics and mutation status of a specific gene.
'''

def all_prot_format_df(cancer_object, all_prot, gene_in = 'PTEN', utils = u):
    mut_type = cancer_object.get_genotype_all_vars(gene_in)
    
    if cancer_object.get_cancer_type() not in ('luad'):
        # Keep only tumor samples from proteomics
        prot_and_mutations = cancer_object.join_omics_to_mutations(
            mutations_genes = [gene_in], omics_df_name = 'proteomics', omics_genes = all_prot, tissue_type = 'tumor')
        # Reduce a multiindex 
        if isinstance(prot_and_mutations.keys(), pd.core.indexes.multi.MultiIndex):
            prot_and_mutations = utils.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
            prot_and_mutations = rename_duplicate_cols(prot_and_mutations)
        
        # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
        mut_type = mut_type[['Mutation']] 
        prot_df = prot_and_mutations.iloc[:,:-4] # Keep only proteomics
        merged = prot_df.join(mut_type)
        
        # Keep only Wildtype and deletion
        compare = ['Wildtype_Tumor','Deletion']
        get = merged['Mutation'].isin(compare)
        mut_wt = merged[get]
    
    
    # Luad has no somatic mutations for PTEN which changes some things
    elif cancer_object.get_cancer_type() in ('luad'): 
        # get_genotype_all_vars add cnv data under the column PTEN
        mut_type = mut_type.drop(columns= gene_in)
        # different code because no somatic mutation data for pten (can't join to somatic mutations)
        omics = cancer_object.join_omics_to_omics(df1_name = 'CNV', df2_name='proteomics',genes1= gene_in, 
            genes2= all_prot, tissue_type = 'tumor')
        omics = utils.reduce_multiindex(omics, levels_to_drop = 1, flatten = True)
        omics = rename_duplicate_cols(omics)
        omics = omics.drop(columns='PTEN_CNV')
       
        # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
        merged = omics.join(mut_type) 
        # Keep only No_Mutation (wildtype) and deletion
        compare = ['No_Mutation','Deletion']
        get = merged['Mutation'].isin(compare)
        mut_wt = merged[get]
        mut_wt['Mutation'] = np.where(
            mut_wt['Mutation'] == 'No_Mutation', 'Wildtype_Tumor', 'Deletion')
    
    mut_wt = mut_wt.rename(columns={col: re.sub(r'_proteomics', '', col) for col in mut_wt.columns.tolist()})
    return mut_wt


In [5]:
'''
Params
cancer_object: Object of the loaded cancer data set.
cancer_name: String. Name to add to the created df.
del_wt_df: DataFrame. df of only samples with PTEN deletion and wildtype. 
all_prot_list: List. All proteins in proteomics df. 

Returns a df with the differential expression between proteomics with PTEN del and wt (del - wt). 
'''

def get_change_in_medians_df(cancer_object, cancer_name, del_wt_df, all_prot_list):
    if  cancer_object.get_cancer_type() in ('endometrial'):
        d = del_wt_df[del_wt_df.Mutation == "Truncation"]
        wt = del_wt_df[del_wt_df.Mutation == "Wildtype_Tumor"]
    else:
        d = del_wt_df[del_wt_df.Mutation == "Deletion"]
        wt = del_wt_df[del_wt_df.Mutation == "Wildtype_Tumor"]
    del_med = d.median()
    wt_med = wt.median()

    med_dict = {}
    
    # Correlation: + is mutant up compared to wt, - is mutant down
    for prot in all_prot_list:
        if prot in del_med.index and prot in wt_med.index:
            dif = del_med[prot] - wt_med[prot]
            med_dict[prot] = dif
        
    df = pd.DataFrame.from_dict(med_dict, orient='index', columns=[cancer_name+'_Median'])
    df = df.reset_index().rename(columns={'index':'Proteomics'})
    
    return df
        

In [6]:
''' Adds '_i' to duplicate col names where the int i increases by one with each duplicate 
Returns a df with unique column names. '''

def rename_duplicate_cols(df):
    cols = pd.Series(df.columns[:])

    for dup in cols[cols.duplicated()].unique(): 
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]

    # rename the columns with the cols list.
    df.columns=cols
    return df

#  Step 1: Create data frames with p-values and differential expressions

Each cancer needs a data frame containing only samples that have PTEN cnv deletions and PTEN wildtype with trans proteomics. Use wrap_ttest to run t-tests for all genes in the proteomic data frame. Use get_change_in_medians_df to create the data frame with differential expression values. Merge both data frames. 

Load in cancer data sets from cptac. 

In [7]:
g = cptac.Gbm()
en = cptac.Endometrial()
h = cptac.Hnscc()
l = cptac.Luad()
ls = cptac.Lscc()
o = cptac.Ovarian()
col = cptac.Colon()
b = cptac.Brca()

# Gbm

Part 1: Format data frame using all_prot_format_df. This returns a dataframe with proteomics and a Mutation column containing only PTEN cnv deletions and wildtype tumors. 

In [8]:
gene = 'PTEN'
g_prot = g.get_proteomics()
g_prot = g_prot.drop(columns = 'PTEN') # cis effect
g_prot_list = list(g_prot.columns)

g_del_wt = all_prot_format_df(g, g_prot_list)
g_del_wt.head()

Name,A1BG,A2M,AAAS,AACS,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,...,ZSWIM8,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3,Mutation
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00104,0.077630,0.487228,-0.254208,-0.144373,0.551881,-0.025276,-0.467451,-0.089511,-0.078806,0.329603,...,-0.047437,-0.105908,-0.347076,NaN,0.459635,0.079452,-0.784983,-0.488441,0.167990,Deletion
C3L-00365,-0.145975,0.798796,0.184242,-0.470603,NaN,0.390211,0.245466,-0.609998,0.118625,-0.086927,...,0.161975,-0.213093,0.235571,NaN,0.107421,0.048724,0.138403,-0.290141,0.405037,Deletion
C3L-00674,0.821991,1.096470,-0.094421,-0.106304,0.084578,0.176402,-0.248151,0.014061,-0.699773,-0.638462,...,-0.065534,-0.306717,0.879991,NaN,0.883564,-0.172222,0.011876,-0.131889,-0.503581,Deletion
C3L-00677,-0.064567,0.129385,0.047751,-0.118187,0.237434,NaN,0.303847,0.322163,-0.555479,-0.363414,...,-0.254535,0.463653,0.580230,0.503044,-0.604986,0.178077,-0.720059,-0.150197,-0.268715,Deletion
C3L-01040,-0.763691,-1.031834,-0.217194,-0.695701,0.184173,-0.474816,-0.051789,0.344842,-0.642746,0.068863,...,-0.092502,0.010639,-0.465079,NaN,-0.500083,0.112651,1.004660,-0.230304,-0.102416,Deletion


Part 2: Run T-tests. Create a data frame of just significant comparisons and another data frame for all comparisons. A moderately stringent correction is used (FDR_BH).

In [9]:
g_cols = list(g_del_wt.columns[:-1])

# Get all pvals
g_pval = u.wrap_ttest(g_del_wt, 'Mutation', g_cols, return_all = True, correction_method = 'fdr_bh', mincount = 5)
g_pval = g_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'GBM_P_Value'})

Part 3: Create the differential expression df. (median of tumors with PTEN wildtype - median of tumors with PTEN cnv deletions)

In [10]:
prot_and_isoforms = list(g_del_wt[:-1])
g_med = get_change_in_medians_df(g, "GBM", g_del_wt, prot_and_isoforms)

Part 4: Merge the p-value and differential expression dfs.

In [11]:
g_merged = g_pval.merge(g_med, on='Proteomics', how='outer')
g_merged = g_merged.replace(to_replace = r'_proteomics', value = '', regex = True)

# Create csv
g_merged.to_csv('csv/Single_Cancer/GBM_pval_medians.csv',index=False)
g_merged.head()

,Proteomics,GBM_P_Value,GBM_Median
0,ARMH3,5.766739e-07,-0.405134
1,CUTC,8.514758e-07,-0.553255
2,CUL2,2.783477e-06,-0.586396
3,PIP4K2A,2.783477e-06,-0.838882
4,GDI2,2.783477e-06,-0.610188


# Repeat for other cancers.

In [12]:
cancer_objects = {'HNSCC':h, 'LUAD':l, 'LSCC':ls, 'BR':b, 'CO':col, 'OV':o}

In [13]:
import warnings
warnings.filterwarnings('ignore')

gene = 'PTEN'
merged_dfs = {}
for cancer in cancer_objects:
    prot = cancer_objects[cancer].get_proteomics()
    prot = prot.drop(columns = 'PTEN') # cis effect
    
    # Reduce a multiindex 
    if isinstance(prot.keys(), pd.core.indexes.multi.MultiIndex):
        prot = u.reduce_multiindex(prot, levels_to_drop = 1)
    prot_list = list(prot.columns)
    
    # Format df for t-test
    del_wt = all_prot_format_df(cancer_objects[cancer], prot_list)
    
    # Get list of unique cols for t-test and get_change_in_medians
    prot_and_iso = list(del_wt.columns[:-1])

    # Get all pvals
    min_num = 5
    all_pval = u.wrap_ttest(del_wt, 'Mutation', prot_and_iso, return_all = True, 
                            correction_method = 'fdr_bh', mincount = min_num)
    all_pval = all_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': cancer+'_P_Value'})
    
    # Get change in medians
    delta_median_df = get_change_in_medians_df(cancer_objects[cancer], cancer, del_wt, prot_and_iso)

    # Merge pval_df and delta_median_df
    pval_medians_df = all_pval.merge(delta_median_df, on='Proteomics', how='outer')
    merged_dfs[cancer] = pval_medians_df # testing purposes
    
    # Create csv
    pval_medians_df.to_csv('csv/Single_Cancer/'+cancer+'_pval_medians.csv', index=False)
    print(pval_medians_df.head(), '\n')

  Proteomics  HNSCC_P_Value  HNSCC_Median
0        NLN       0.000012      0.431666
1     AKR1C3       0.000054      0.951905
2     AKR1C1       0.000240      1.961262
3       ADI1       0.000416      0.513198
4      EPHX1       0.000678      0.871688 

  Proteomics  LUAD_P_Value  LUAD_Median
0       ATE1      0.003010     -0.88365
1      SHOC2      0.003209     -0.60600
2       EGFR      0.003209      2.32970
3     LANCL2      0.004599      1.21475
4    CWF19L1      0.005583     -0.57265 

  Proteomics  LSCC_P_Value  LSCC_Median
0       ATE1      0.006134     -0.63115
1      ATAD1      0.010108     -1.01335
2      INTS2      0.013476      0.34010
3     FAM45A      0.013476     -0.36335
4       CCT7      0.013476      0.55675 

  Proteomics  BR_P_Value  BR_Median
0     TMSB10    0.131051    0.87630
1       DRG1    0.131051    0.75135
2      EIF4H    0.131051    0.41295
3      RIPK2    0.131051    0.82670
4      MIEF1    0.131051    0.84135 

  Proteomics  CO_P_Value  CO_Median
0       

# Endometrial

The Endometrial data set does not have enough cnv deletions to perform a t-test, however the data set does have enough truncation type mutations (nonsense and frameshift). Different code is needed to create the data frame for Endometrial.

In [14]:
gene = 'PTEN'
prot = en.get_proteomics()
prot = prot.drop(columns = 'PTEN')
e_prot_list = list(prot.columns)

# Get mutations based on priority filter when a sample has multiple mutations
mut_type = en.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = en.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = e_prot_list, tissue_type = 'tumor') # drop Normal samples

prot_df = prot_and_mutations.iloc[:,:-4] #drop Mutation and Location cols
mut_type = mut_type[['Mutation']] # Get Mutation col that includes CNV
merged = prot_df.join(mut_type) # merge 

# Create Truncation category and keep truncation and wt
compare = ['Wildtype_Tumor','Nonsense_Mutation','Frame_Shift_Ins','Frame_Shift_Del']
get = merged['Mutation'].isin(compare)
trunc_wt = merged[get]
trunc_wt['Mutation'] = np.where(
                trunc_wt['Mutation'] == 'Wildtype_Tumor', 'Wildtype_Tumor', 'Truncation')

In [15]:
e_cols = list(trunc_wt.columns[:-1])

# Get all pvals
e_pval = u.wrap_ttest(trunc_wt, 'Mutation', e_cols, return_all = True, correction_method = 'fdr_bh', mincount = 5)
e_pval = e_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'EC_P_Value'})

Differential expression with adaption for trunctation mutations.

In [16]:
t = trunc_wt[trunc_wt.Mutation == "Truncation"]
wt = trunc_wt[trunc_wt.Mutation == "Wildtype_Tumor"]
trunc_med = t.median()
wt_med = wt.median()

en_d = {}

for prot in e_cols:
    if prot in trunc_med.index and prot in wt_med.index:
        dif = trunc_med[prot] - wt_med[prot]
        en_d[prot] = dif

en_med = pd.DataFrame.from_dict(en_d, orient='index', columns=['EC_Median'])
en_med = en_med.reset_index().rename(columns={'index':'Proteomics'})

In [17]:
e_merged = e_pval.merge(en_med, on='Proteomics',how='outer')
e_merged = e_merged.replace(to_replace = r'_proteomics', value = '', regex = True)

# Create csv
e_merged.to_csv('csv/Single_Cancer/EC_pval_medians.csv',index=False)
e_merged.head()

,Proteomics,EC_P_Value,EC_Median
0,NOL10,0.000915,-0.44200
1,ABT1,0.001242,-0.48510
2,TOPBP1,0.001242,-0.61795
3,UTP25,0.001242,-0.73450
4,RIF1,0.001590,-0.54640


# Get number of significant genes in each cancer

In [18]:
# Add Gbm and En to merged_dfs dictionary
merged_dfs['GBM'] = g_merged
merged_dfs['EC'] = e_merged

In [19]:
# Using corrected pvals from wrap_ttest, each cancer's cutoff is 0.05
all_sig = []

for cancer in merged_dfs:
    df = merged_dfs[cancer]
    sig_df = df.loc[df[cancer+'_P_Value'] < 0.05]
    print(cancer, 'sig comparisons:', len(sig_df))
    sig_list = list(sig_df['Proteomics'])
    all_sig.append(sig_list)
        

flat_list = [item for sublist in all_sig for item in sublist] #change list of lists, to just one list
sig = list(set(flat_list)) # keep only one if gene in multiple cancer sig lists
print('\nNumber of significant comparisons in at least 1 cancer:', len(sig))

HNSCC sig comparisons: 570
LUAD sig comparisons: 111
LSCC sig comparisons: 57
BR sig comparisons: 0
CO sig comparisons: 0
OV sig comparisons: 1
GBM sig comparisons: 1900
EC sig comparisons: 415

Number of significant comparisons in at least 1 cancer: 2658


# Get a list of significant genes in at least one cancer

In [20]:
s = pd.Series(sig)
print('Number of significant genes in at least one cancer:', len(s))
s.to_csv('csv/list_sig_one_cancer.csv', index=False)

Number of significant genes in at least one cancer: 2658


# Get a list of significant genes in multiple cancers

In [21]:
# sig in multiple cancers
mult = list(set(i for i in flat_list if flat_list.count(i) > 1)) # Keep genes sig in more than 1 cancer
print('Number of significant genes in mult cancers:', len(mult))

Number of significant genes in mult cancers: 348


In [22]:
m = pd.Series(mult)
m = m.replace(to_replace = r'_proteomics$', value = '', regex = True)
m.to_csv('csv/list_sig_multiple_cancers.csv', index=False)